In [1]:
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model

import numpy as np
import json
import os
import pickle

import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

Using TensorFlow backend.
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
input_dim = (28,28,1)
encoder_conv_filters = [32,64,64,64]
encoder_conv_kernel_size = [3,3,3,3]
encoder_conv_strides = [1,2,2,1]
decoder_conv_t_filters = [64,64,32,1]
decoder_conv_t_kernel_size = [3,3,3,3]
decoder_conv_t_strides = [1,2,2,1]
n_layers_encoder = len(encoder_conv_filters)
z_dim = 2
n_layers_decoder = len(decoder_conv_t_filters)

In [3]:
(x_train, y_train), (x_test, y_test) = load_mnist()

### THE ENCODER

In [4]:
### THE ENCODER
encoder_input = Input(shape=input_dim, name='encoder_input')

x = encoder_input

for i in range(n_layers_encoder):
    conv_layer = Conv2D(
        filters = encoder_conv_filters[i]
        , kernel_size = encoder_conv_kernel_size[i]
        , strides = encoder_conv_strides[i]
        , padding = 'same'
        , name = 'encoder_conv_' + str(i)
        )

    x = conv_layer(x)
    x = LeakyReLU()(x)

shape_before_flattening = K.int_shape(x)[1:]

x = Flatten()(x)
encoder_output= Dense(z_dim, name='encoder_output')(x)

encoder = Model(encoder_input, encoder_output)


### THE DECODER
decoder_input = Input(shape=(z_dim,), name='decoder_input')

x = Dense(np.prod(shape_before_flattening))(decoder_input)
x = Reshape(shape_before_flattening)(x)

for i in range(n_layers_decoder):
    conv_t_layer = Conv2DTranspose(
        filters = decoder_conv_t_filters[i]
        , kernel_size = decoder_conv_t_kernel_size[i]
        , strides = decoder_conv_t_strides[i]
        , padding = 'same'
        , name = 'decoder_conv_t_' + str(i)
        )

    x = conv_t_layer(x)

    if i < n_layers_decoder - 1:
        x = LeakyReLU()(x)

    else:
        x = Activation('sigmoid')(x)

decoder_output = x

decoder = Model(decoder_input, decoder_output)

### THE FULL AUTOENCODER
model_input = encoder_input
model_output = decoder(encoder_output)

model = Model(model_input, model_output)

Instructions for updating:
Colocations handled automatically by placer.


### THE FULL AUTOENCODER

In [5]:
model_input = encoder_input
model_output = decoder(encoder_output)

model = Model(model_input, model_output)

In [9]:
optimizer = Adam(lr=0.0001)

def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis = [1,2,3])

model.compile(optimizer=optimizer, loss = r_loss)

model.fit(     
    x = x_train,
    y = x_train,
    batch_size = 32,
    shuffle = True,
    epochs = 200
)

Epoch 1/200
 1152/60000 [..............................] - ETA: 2:57 - loss: 0.1659

KeyboardInterrupt: 